## Step 1: Install Required Packages

First, we need to install CrewAI and its dependencies. We'll use:
- `crewai`: The main framework
- `crewai-tools`: Pre-built tools for agents

In [1]:
# Install required packages
!pip install -q crewai crewai-tools

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.20.3 which is incompatible.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Step 2: Setup API Keys and Environment

To use OpenAI's GPT-4.1-mini model, you need an API key and base URL.

**Security Best Practice:** Store your API key in a `.env` file, not in the code!

In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

if "OPENAI_BASE_URL" not in os.environ:
    os.environ["OPENAI_BASE_URL"] = getpass.getpass("Enter your OpenAI Base URL: ")

## Step 3: Create Custom Tools

Tools are functions that agents can use to perform specific tasks. Let's create custom tools for our movie production assistant.

### Tool 1: Script Sentiment Analyzer
Analyzes the emotional tone of a script excerpt.

In [3]:
from crewai.tools import tool
import re

@tool("Script Sentiment Analyzer")
def analyze_script_sentiment(script_text: str) -> str:
    """
    Analyzes the sentiment and emotional tone of a movie script excerpt.
    Returns sentiment score and dominant emotions.

    Args:
        script_text: A string containing the script excerpt to analyze
    """
    # Simple sentiment analysis based on keywords
    positive_words = ['love', 'happy', 'joy', 'wonderful', 'amazing', 'brilliant', 'triumph', 'success']
    negative_words = ['hate', 'sad', 'angry', 'terrible', 'awful', 'tragedy', 'failure', 'death']
    suspense_words = ['mystery', 'unknown', 'hidden', 'secret', 'suspense', 'thriller']

    script_lower = script_text.lower()

    pos_count = sum(1 for word in positive_words if word in script_lower)
    neg_count = sum(1 for word in negative_words if word in script_lower)
    sus_count = sum(1 for word in suspense_words if word in script_lower)

    total = pos_count + neg_count + sus_count

    if total == 0:
        return "Sentiment: Neutral - No strong emotional indicators detected."

    result = f"Sentiment Analysis Results:\n"
    result += f"- Positive tone: {pos_count}/{total} ({pos_count/total*100:.1f}%)\n"
    result += f"- Negative tone: {neg_count}/{total} ({neg_count/total*100:.1f}%)\n"
    result += f"- Suspenseful tone: {sus_count}/{total} ({sus_count/total*100:.1f}%)\n"

    if pos_count > neg_count and pos_count > sus_count:
        result += "\nOverall: Uplifting and positive story"
    elif neg_count > pos_count and neg_count > sus_count:
        result += "\nOverall: Dark and dramatic narrative"
    else:
        result += "\nOverall: Tense and mysterious atmosphere"

    return result

# Test the tool
test_script = "A tale of love and triumph as Sarah discovers the hidden mystery that brings her joy."
print(analyze_script_sentiment.run(test_script))

Sentiment Analysis Results:
- Positive tone: 3/5 (60.0%)
- Negative tone: 0/5 (0.0%)
- Suspenseful tone: 2/5 (40.0%)

Overall: Uplifting and positive story


### Tool 2: Budget Calculator
Calculates production costs based on different categories.

In [4]:
@tool("Production Budget Calculator")
def calculate_production_budget(category: str, days: int = 30, crew_size: int = 50) -> str:
    """
    Calculates estimated production budget for different movie categories.

    Args:
        category: Type of production (indie, medium, blockbuster)
        days: Number of shooting days
        crew_size: Number of crew members
    """
    # Base daily rates per category
    rates = {
        'indie': {'daily': 50000, 'crew': 500, 'equipment': 10000},
        'medium': {'daily': 200000, 'crew': 2000, 'equipment': 50000},
        'blockbuster': {'daily': 1000000, 'crew': 10000, 'equipment': 250000}
    }

    category = category.lower()
    if category not in rates:
        return f"Unknown category. Choose from: {', '.join(rates.keys())}"

    rate = rates[category]

    production_cost = rate['daily'] * days
    crew_cost = rate['crew'] * crew_size * days
    equipment_cost = rate['equipment'] * days
    contingency = (production_cost + crew_cost + equipment_cost) * 0.15

    total = production_cost + crew_cost + equipment_cost + contingency

    result = f"📊 Budget Breakdown for {category.upper()} Production:\n"
    result += f"\n🎬 Production Costs: ${production_cost:,}"
    result += f"\n👥 Crew Costs ({crew_size} members): ${crew_cost:,}"
    result += f"\n📹 Equipment Rental: ${equipment_cost:,}"
    result += f"\n🛡️ Contingency (15%): ${contingency:,}"
    result += f"\n\n💰 TOTAL ESTIMATED BUDGET: ${total:,}"
    result += f"\n📅 For {days} shooting days"

    return result

# Test the tool
print(calculate_production_budget.run("medium", days=45, crew_size=75))

📊 Budget Breakdown for MEDIUM Production:

🎬 Production Costs: $9,000,000
👥 Crew Costs (75 members): $6,750,000
📹 Equipment Rental: $2,250,000
🛡️ Contingency (15%): $2,700,000.0

💰 TOTAL ESTIMATED BUDGET: $20,700,000.0
📅 For 45 shooting days


### Tool 3: Soundtrack Genre Recommender
Suggests music genres based on the movie's theme and mood.

In [5]:
@tool("Soundtrack Genre Recommender")
def recommend_soundtrack_genre(movie_genre: str, mood: str) -> str:
    """
    Recommends soundtrack genres based on movie genre and mood.

    Args:
        movie_genre: The genre of the movie (action, drama, comedy, horror, sci-fi, romance)
        mood: The desired mood (intense, light, emotional, mysterious, upbeat)
    """
    recommendations = {
        'action': {
            'intense': ['Epic Orchestral', 'Electronic/Synth', 'Heavy Metal'],
            'light': ['Pop Rock', 'Funk', 'Electronic Pop'],
            'emotional': ['Cinematic Strings', 'Piano & Strings', 'Alternative Rock'],
        },
        'drama': {
            'intense': ['Classical', 'Dramatic Orchestral', 'Jazz Noir'],
            'emotional': ['Piano Solo', 'String Quartet', 'Acoustic Guitar'],
            'mysterious': ['Ambient', 'Minimalist Piano', 'Contemporary Classical'],
        },
        'comedy': {
            'upbeat': ['Jazz', 'Pop', 'Ska', 'Funk'],
            'light': ['Acoustic Pop', 'Ukulele', 'Whistling & Bells'],
        },
        'horror': {
            'intense': ['Dark Ambient', 'Industrial', 'Atonal Orchestral'],
            'mysterious': ['Theremin', 'Dissonant Strings', 'Electronic Drone'],
        },
        'sci-fi': {
            'intense': ['Electronic/Synth', 'Cyberpunk', 'Orchestral Hybrid'],
            'mysterious': ['Ambient Electronic', 'Experimental', 'Synth Pad'],
        },
        'romance': {
            'emotional': ['Classical Piano', 'Acoustic', 'Indie Folk'],
            'light': ['Soft Pop', 'Bossa Nova', 'Acoustic Guitar'],
            'upbeat': ['Pop', 'Soul', 'Light Jazz'],
        }
    }

    genre = movie_genre.lower()
    mood_key = mood.lower()

    if genre not in recommendations:
        return f"Unknown genre. Try: {', '.join(recommendations.keys())}"

    genre_moods = recommendations[genre]
    if mood_key not in genre_moods:
        return f"For {genre}, available moods are: {', '.join(genre_moods.keys())}"

    suggested = genre_moods[mood_key]

    result = f"🎵 Soundtrack Recommendations for {genre.upper()} ({mood} mood):\n\n"
    for i, genre_rec in enumerate(suggested, 1):
        result += f"{i}. {genre_rec}\n"

    result += f"\n💡 Tip: Consider mixing {suggested[0]} with {suggested[-1]} for dynamic scenes!"

    return result

# Test the tool
print(recommend_soundtrack_genre.run("sci-fi", "mysterious"))

🎵 Soundtrack Recommendations for SCI-FI (mysterious mood):

1. Ambient Electronic
2. Experimental
3. Synth Pad

💡 Tip: Consider mixing Ambient Electronic with Synth Pad for dynamic scenes!


## Step 4: Integrate MCP Tool (Model Context Protocol)

MCP tools allow agents to interact with external systems and APIs. Here we'll create a simulated MCP tool for accessing a movie database.

**Note:** In production, MCP tools connect to real external services. This is a simplified demonstration.

In [6]:
from typing import Dict, Any

@tool("Movie Market Research MCP")
def fetch_market_data(genre: str) -> str:
    """
    Fetches market research data for a specific movie genre from the database.
    This simulates an MCP tool that connects to external market research APIs.

    Args:
        genre: The movie genre to research (action, drama, comedy, horror, sci-fi, romance)
    """
    # Simulated market data (in production, this would call a real API)
    market_database = {
        'action': {
            'avg_box_office': '$450M',
            'audience_demographic': '18-35 years, 65% male',
            'peak_season': 'Summer (May-August)',
            'trending_themes': ['Superheroes', 'International espionage', 'Disaster scenarios'],
            'streaming_performance': 'High - 85% completion rate',
        },
        'drama': {
            'avg_box_office': '$120M',
            'audience_demographic': '30-60 years, 55% female',
            'peak_season': 'Awards season (October-February)',
            'trending_themes': ['Social justice', 'Biography', 'Family dynamics'],
            'streaming_performance': 'Medium - 72% completion rate',
        },
        'comedy': {
            'avg_box_office': '$180M',
            'audience_demographic': '18-45 years, balanced gender',
            'peak_season': 'Year-round, slight spike in holidays',
            'trending_themes': ['Romantic comedy', 'Workplace humor', 'Cultural clash'],
            'streaming_performance': 'Very High - 90% completion rate',
        },
        'horror': {
            'avg_box_office': '$95M',
            'audience_demographic': '16-30 years, 52% male',
            'peak_season': 'Fall (September-November)',
            'trending_themes': ['Psychological horror', 'Folk horror', 'Home invasion'],
            'streaming_performance': 'High - 82% completion rate',
        },
        'sci-fi': {
            'avg_box_office': '$380M',
            'audience_demographic': '18-40 years, 60% male',
            'peak_season': 'Summer and Holiday season',
            'trending_themes': ['AI & technology', 'Space exploration', 'Time travel'],
            'streaming_performance': 'High - 88% completion rate',
        },
        'romance': {
            'avg_box_office': '$85M',
            'audience_demographic': '18-50 years, 70% female',
            'peak_season': 'Valentine\'s Day season (January-February)',
            'trending_themes': ['Second chance romance', 'Holiday romance', 'Cross-cultural love'],
            'streaming_performance': 'Very High - 92% completion rate',
        },
    }

    genre_key = genre.lower()

    if genre_key not in market_database:
        return f"❌ No market data available for '{genre}'. Available genres: {', '.join(market_database.keys())}"

    data = market_database[genre_key]

    result = f"📈 Market Research Report: {genre.upper()} Genre\n"
    result += f"\n💰 Average Box Office: {data['avg_box_office']}"
    result += f"\n👥 Target Demographic: {data['audience_demographic']}"
    result += f"\n📅 Best Release Window: {data['peak_season']}"
    result += f"\n🎬 Trending Themes:"
    for theme in data['trending_themes']:
        result += f"\n   • {theme}"
    result += f"\n📺 Streaming Performance: {data['streaming_performance']}"
    result += f"\n\n✅ Data retrieved from Market Intelligence Database (MCP)"

    return result

# Test the MCP tool
print(fetch_market_data.run("action"))

📈 Market Research Report: ACTION Genre

💰 Average Box Office: $450M
👥 Target Demographic: 18-35 years, 65% male
📅 Best Release Window: Summer (May-August)
🎬 Trending Themes:
   • Superheroes
   • International espionage
   • Disaster scenarios
📺 Streaming Performance: High - 85% completion rate

✅ Data retrieved from Market Intelligence Database (MCP)


## Step 5: Configure the Language Model

Now let's set up OpenAI's GPT-4.1-mini model to power our agents. This model is fast and cost-effective, perfect for agent-based workflows.

In [7]:
from crewai import LLM

# Initialize GPT-4.1-mini using CrewAI's LLM wrapper
llm = LLM(
    model="openai/gpt-4.1-mini",
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_BASE_URL"],
    temperature=0.7,  # Controls creativity (0=focused, 1=creative)
)

print("✅ Language model configured: GPT-4.1-mini")
print("   - Model: openai/gpt-4.1-mini")
print("   - Temperature: 0.7 (balanced creativity)")

✅ Language model configured: GPT-4.1-mini
   - Model: openai/gpt-4.1-mini
   - Temperature: 0.7 (balanced creativity)


## Step 6: Create Specialized Agents

Agents are AI assistants with specific roles and tools. Let's create a crew of specialized agents for movie production:

1. **Script Analyst** - Analyzes scripts and provides creative feedback
2. **Budget Manager** - Handles financial planning and budgeting
3. **Market Researcher** - Provides market insights and recommendations

In [8]:
from crewai import Agent

# Agent 1: Script Analyst
script_analyst = Agent(
    role="Script Analyst",
    goal="Analyze movie scripts for sentiment, themes, and recommend appropriate soundtracks",
    backstory="""You are an experienced script analyst with 15 years in Hollywood.
    You have a keen eye for emotional depth and understand how music enhances storytelling.
    You've worked on numerous award-winning films and know what makes a script compelling.""",
    tools=[analyze_script_sentiment, recommend_soundtrack_genre],
    llm=llm,
    verbose=True,  # Shows detailed thinking process
    allow_delegation=False,  # Can't delegate tasks to other agents
)

# Agent 2: Budget Manager
budget_manager = Agent(
    role="Production Budget Manager",
    goal="Calculate accurate production budgets and optimize spending across departments",
    backstory="""You are a meticulous financial expert specializing in film production.
    With an MBA and 10 years of experience managing budgets for indie to blockbuster films,
    you ensure projects stay financially viable while maintaining creative vision.""",
    tools=[calculate_production_budget],
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

# Agent 3: Market Researcher
market_researcher = Agent(
    role="Film Market Research Analyst",
    goal="Provide data-driven insights about market trends, audience preferences, and optimal release strategies",
    backstory="""You are a data scientist specializing in entertainment industry analytics.
    You analyze box office trends, streaming data, and audience demographics to help studios
    make informed decisions about production and distribution.""",
    tools=[fetch_market_data],
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

print("✅ Created 3 specialized agents:")
print("   1. Script Analyst (sentiment & soundtrack)")
print("   2. Budget Manager (financial planning)")
print("   3. Market Researcher (market intelligence)")

✅ Created 3 specialized agents:
   1. Script Analyst (sentiment & soundtrack)
   2. Budget Manager (financial planning)
   3. Market Researcher (market intelligence)


## Step 7: Define Tasks for Agents

Tasks are specific assignments given to agents. Each task has:
- A clear description
- An assigned agent
- Expected output format

In [9]:
from crewai import Task

# Sample movie concept for our tasks
movie_concept = """
Title: "Echoes of Tomorrow"
Genre: Sci-Fi Thriller
Logline: A brilliant AI researcher discovers a hidden message in quantum data that reveals
humanity's tragic future. Racing against time and a mysterious organization, she must decide
whether to alter the timeline or accept humanity's fate.

Script Excerpt: "In the depths of the unknown laboratory, Sarah's discovery brings both
triumph and terror. The hidden secret she uncovers could save millions, but the mystery
of who left this message haunts her every waking moment."
"""

# Task 1: Analyze the script
task_analyze_script = Task(
    description=f"""Analyze the following movie concept and script excerpt:

    {movie_concept}

    Use your tools to:
    1. Analyze the sentiment of the script excerpt
    2. Recommend appropriate soundtrack genres based on the genre and mood
    3. Provide creative suggestions for enhancing the emotional impact
    """,
    expected_output="A detailed analysis including sentiment breakdown, soundtrack recommendations, and creative suggestions",
    agent=script_analyst,
)

# Task 2: Calculate budget
task_budget_calculation = Task(
    description="""Based on the sci-fi thriller movie concept, calculate the production budget.

    Assume:
    - This is a MEDIUM budget production
    - 60 days of shooting
    - Crew size of 85 people

    Provide a detailed breakdown and recommendations for budget optimization.
    """,
    expected_output="Complete budget breakdown with optimization recommendations",
    agent=budget_manager,
)

# Task 3: Market research
task_market_research = Task(
    description="""Conduct market research for the sci-fi thriller genre.

    Use the MCP tool to fetch market data and provide:
    1. Current market performance of sci-fi films
    2. Target audience insights
    3. Optimal release timing
    4. Strategic recommendations based on trending themes
    """,
    expected_output="Comprehensive market analysis with strategic release recommendations",
    agent=market_researcher,
)

print("✅ Created 3 tasks:")
print("   1. Script Analysis Task")
print("   2. Budget Calculation Task")
print("   3. Market Research Task")

✅ Created 3 tasks:
   1. Script Analysis Task
   2. Budget Calculation Task
   3. Market Research Task


## Step 8: Assemble the Crew and Execute

Now we bring it all together! A Crew coordinates multiple agents working on related tasks.

**Process Types:**
- `sequential`: Tasks execute one after another (used here)
- `hierarchical`: A manager agent coordinates other agents

In [ ]:
from crewai import Crew, Process

# Assemble the crew
movie_production_crew = Crew(
    agents=[script_analyst, budget_manager, market_researcher],
    tasks=[task_analyze_script, task_budget_calculation, task_market_research],
    process=Process.sequential,  # Tasks run in order
    verbose=True,  # Enable detailed output for learning
)

print("✅ Crew assembled with 3 agents and 3 tasks")
print("\n🚀 Starting production analysis...\n")
print("=" * 80)

# Execute the crew's tasks
result = movie_production_crew.kickoff()

print("\n" + "=" * 80)
print("\n✅ Analysis Complete!\n")

✅ Crew assembled with 3 agents and 3 tasks

🚀 Starting production analysis...



╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d615c921-a786-44c7-bca8-e367136c146b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Analyze the following movie concept and script excerpt:                                                  │
│                                                                                                                 │
│                                                                                                                 │
│  Title: "Echoes of Tomorrow"                                                                                    │
│  Genre: Sci-Fi Thriller                                                                                         │
│  Logline: A brilliant AI researcher discovers a hidden message in quantum data that reveals                     │
│  humanity's tragic future. Racing against time and a mysterious organization, she must decide                   │
│  whether to alter the timeline or accept humanity's fate.                                                       │
│                                                                                                                 │
│  Script Excerpt: "In the depths of the unknown laboratory, Sarah's discovery brings both                        │
│  triumph and terror. The hidden secret she uncovers could save millions, but the mystery                        │
│  of who left this message haunts her every waking moment."                                                      │
│                                                                                                                 │
│                                                                                                                 │
│      Use your tools to:                                                                                         │
│      1. Analyze the sentiment of the script excerpt                                                             │
│      2. Recommend appropriate soundtrack genres based on the genre and mood                                     │
│      3. Provide creative suggestions for enhancing the emotional impact                                         │
│                                                                                                                 │
│  ID: b8005f01-f497-46ea-865c-c5a1263889e5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Script Analyst                                                                                          │
│                                                                                                                 │
│  Task: Analyze the following movie concept and script excerpt:                                                  │
│                                                                                                                 │
│                                                                                                                 │
│  Title: "Echoes of Tomorrow"                                                                                    │
│  Genre: Sci-Fi Thriller                                                                                         │
│  Logline: A brilliant AI researcher discovers a hidden message in quantum data that reveals                     │
│  humanity's tragic future. Racing against time and a mysterious organization, she must decide                   │
│  whether to alter the timeline or accept humanity's fate.                                                       │
│                                                                                                                 │
│  Script Excerpt: "In the depths of the unknown laboratory, Sarah's discovery brings both                        │
│  triumph and terror. The hidden secret she uncovers could save millions, but the mystery                        │
│  of who left this message haunts her every waking moment."                                                      │
│                                                                                                                 │
│                                                                                                                 │
│      Use your tools to:                                                                                         │
│      1. Analyze the sentiment of the script excerpt                                                             │
│      2. Recommend appropriate soundtrack genres based on the genre and mood                                     │
│      3. Provide creative suggestions for enhancing the emotional impact                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🔧 Tool Execution Started (#1) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: script_sentiment_analyzer                                                                                │
│  Args: {'script_text': "In the depths of the unknown laboratory, Sarah's discovery brings both triumph and      │
│  terror. The hidden secret she uncovers could save millions, but the mystery of who left this message...        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d615c921-a786-44c7-bca8-e367136c146b                                                                       │
│  Final Output: Comprehensive Market Analysis and Strategic Release Recommendations for Sci-Fi Thriller Genre:   │
│                                                                                                                 │
│  1. Current Market Performance of Sci-Fi Films:                                                                 │
│  The sci-fi genre demonstrates strong market performance with an average box office revenue of $380 million,    │
│  indicating significant audience interest and commercial viability. Streaming data shows a high engagement      │
│  level with an 88% completion rate, reflecting sustained viewer attention and positive reception on digital     │
│  platforms.                                                                                                     │
│                                                                                                                 │
│  2. Target Audience Insights:                                                                                   │
│  The primary demographic for sci-fi films is adults aged 18-40, with a dominant male viewership comprising 60%  │
│  of the audience. This suggests marketing strategies should focus on platforms and messaging that resonate      │
│  with this age group and gender, while also considering ways to broaden appeal to female viewers for expanded   │
│  reach.                                                                                                         │
│                                                                                                                 │
│  3. Optimal Release Timing:                                                                                     │
│  The ideal release windows for sci-fi films are during the summer months and the holiday season. These periods  │
│  align with peak movie-going times, maximizing box office potential and audience availability. Scheduling a     │
│  sci-fi thriller release during these times can capitalize on favorable market conditions.                      │
│                                                                                                                 │
│  4. Strategic Recommendations Based on Trending Themes:                                                         │
│  Current trending themes in the sci-fi genre include artificial intelligence and technology, space              │
│  exploration, and time travel. Integrating these elements into the narrative or marketing can enhance           │
│  relevance and audience interest. For "Echoes of Tomorrow," emphasizing its mysterious and suspenseful tone     │
│  alongside themes of technological discovery aligns well with market trends.                                    │
│                                                                                                                 │
│  Summary:                                                                                                       │
│  To optimize the success of "Echoes of Tomorrow," position it as a summer or holiday release targeting the      │
│  18-40 demographic with a marketing focus that highlights its intense sci-fi thriller elements. Incorporate     │
│  trending themes like AI and technology in promotional materials to attract the core sci-fi audience while      │
│  leveraging high streaming completion rates by plannin



✅ Analysis Complete!



╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are available!                                │
│                                                                             │
│  View insights including:                                                   │
│    • Agent decision-making process                                          │
│    • Task execution flow and timing                                         │
│    • Tool usage details                                                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
Would you like to view your execution traces? [y/N] (20s timeout): 

╭─────────────────────────────────────────── Tracing Preference Saved ────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing has been disabled.                                                                               │
│                                                                                                                 │
│  Your preference has been saved. Future Crew/Flow executions will not collect traces.                           │
│                                                                                                                 │
│  To enable tracing later, do any one of these:                                                                  │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Step 9: View Final Results

Let's examine the consolidated output from all agents.

In [11]:
print("📋 FINAL PRODUCTION REPORT")
print("=" * 80)
print(result)
print("=" * 80)

📋 FINAL PRODUCTION REPORT
Comprehensive Market Analysis and Strategic Release Recommendations for Sci-Fi Thriller Genre:

1. Current Market Performance of Sci-Fi Films:
The sci-fi genre demonstrates strong market performance with an average box office revenue of $380 million, indicating significant audience interest and commercial viability. Streaming data shows a high engagement level with an 88% completion rate, reflecting sustained viewer attention and positive reception on digital platforms.

2. Target Audience Insights:
The primary demographic for sci-fi films is adults aged 18-40, with a dominant male viewership comprising 60% of the audience. This suggests marketing strategies should focus on platforms and messaging that resonate with this age group and gender, while also considering ways to broaden appeal to female viewers for expanded reach.

3. Optimal Release Timing:
The ideal release windows for sci-fi films are during the summer months and the holiday season. These periods

## Step 10: Implementing Observability

Observability helps you monitor and debug your agents. Let's add logging and tracking capabilities.

**Key Observability Features:**
1. Execution time tracking
2. Tool usage monitoring
3. Error logging
4. Agent decision tracking

In [12]:
import time
import json
from datetime import datetime

class CrewObserver:
    """Custom observability class for monitoring CrewAI execution"""

    def __init__(self):
        self.logs = []
        self.start_time = None
        self.end_time = None

    def log_event(self, event_type: str, agent_name: str, details: dict):
        """Log an event during crew execution"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'event_type': event_type,
            'agent': agent_name,
            'details': details
        }
        self.logs.append(log_entry)

    def start_monitoring(self):
        """Start monitoring crew execution"""
        self.start_time = time.time()
        self.log_event('CREW_START', 'System', {'message': 'Crew execution started'})

    def end_monitoring(self):
        """End monitoring and calculate metrics"""
        self.end_time = time.time()
        duration = self.end_time - self.start_time
        self.log_event('CREW_END', 'System', {
            'message': 'Crew execution completed',
            'duration_seconds': round(duration, 2)
        })

    def get_metrics(self):
        """Generate execution metrics"""
        if not self.end_time:
            return "Monitoring still in progress"

        duration = self.end_time - self.start_time
        agent_events = {}

        for log in self.logs:
            agent = log['agent']
            if agent != 'System':
                agent_events[agent] = agent_events.get(agent, 0) + 1

        metrics = f"""\n📊 EXECUTION METRICS\n{'='*60}
⏱️  Total Duration: {duration:.2f} seconds
📝 Total Events Logged: {len(self.logs)}
🤖 Agent Activity:"""

        for agent, count in agent_events.items():
            metrics += f"\n   • {agent}: {count} events"

        return metrics

    def display_logs(self, filter_agent=None):
        """Display execution logs"""
        print(f"\n📋 EXECUTION LOGS ({len(self.logs)} entries)")
        print("=" * 80)

        for log in self.logs:
            if filter_agent and log['agent'] != filter_agent:
                continue

            timestamp = log['timestamp'].split('T')[1].split('.')[0]
            print(f"[{timestamp}] {log['event_type']:15} | {log['agent']:25} | {log['details']}")

# Initialize observer
observer = CrewObserver()
print("✅ Observability system initialized")

✅ Observability system initialized


## Step 11: Run Crew with Observability

Now let's run our crew again with full observability enabled.

In [13]:
# Create a new task with observability hooks
observer.start_monitoring()

# Log agent initialization
observer.log_event('AGENT_INIT', 'Script Analyst', {'tools': ['sentiment_analyzer', 'soundtrack_recommender']})
observer.log_event('AGENT_INIT', 'Budget Manager', {'tools': ['budget_calculator']})
observer.log_event('AGENT_INIT', 'Market Researcher', {'tools': ['market_data_mcp']})

# New movie concept for second run
new_concept = """
Title: "The Last Symphony"
Genre: Drama
Logline: An elderly composer, losing her memory to dementia, races to complete her
final masterpiece while her estranged daughter learns to understand her through music.

Script Excerpt: "Each note brings both joy and sadness as Margaret's fingers dance across
the piano keys. The wonderful melody emerges from the fog of her fading memory, a
triumph over the terrible disease that threatens to steal her life's work."
"""

# Create new task
observed_task = Task(
    description=f"""Analyze this movie concept:
    {new_concept}

    Perform sentiment analysis and recommend soundtrack genres.""",
    expected_output="Sentiment analysis and soundtrack recommendations",
    agent=script_analyst,
)

# Log task start
observer.log_event('TASK_START', 'Script Analyst', {'task': 'Script analysis'})

# Create a smaller crew for demonstration
observed_crew = Crew(
    agents=[script_analyst],
    tasks=[observed_task],
    process=Process.sequential,
    verbose=1,  # Less verbose for cleaner output
)

print("🔍 Running crew with observability enabled...\n")

# Execute
result_observed = observed_crew.kickoff()

# Log completion
observer.log_event('TASK_COMPLETE', 'Script Analyst', {'status': 'success'})
observer.end_monitoring()

print("\n✅ Execution complete with observability tracking!")

🔍 Running crew with observability enabled...



╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 36ec3902-c762-4451-b5ca-4fe30fbe35c6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Analyze this movie concept:                                                                              │
│                                                                                                                 │
│  Title: "The Last Symphony"                                                                                     │
│  Genre: Drama                                                                                                   │
│  Logline: An elderly composer, losing her memory to dementia, races to complete her                             │
│  final masterpiece while her estranged daughter learns to understand her through music.                         │
│                                                                                                                 │
│  Script Excerpt: "Each note brings both joy and sadness as Margaret's fingers dance across                      │
│  the piano keys. The wonderful melody emerges from the fog of her fading memory, a                              │
│  triumph over the terrible disease that threatens to steal her life's work."                                    │
│                                                                                                                 │
│                                                                                                                 │
│      Perform sentiment analysis and recommend soundtrack genres.                                                │
│  ID: 34ae46dc-296f-449d-83aa-5d48c4d3fd36                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Script Analyst                                                                                          │
│                                                                                                                 │
│  Task: Analyze this movie concept:                                                                              │
│                                                                                                                 │
│  Title: "The Last Symphony"                                                                                     │
│  Genre: Drama                                                                                                   │
│  Logline: An elderly composer, losing her memory to dementia, races to complete her                             │
│  final masterpiece while her estranged daughter learns to understand her through music.                         │
│                                                                                                                 │
│  Script Excerpt: "Each note brings both joy and sadness as Margaret's fingers dance across                      │
│  the piano keys. The wonderful melody emerges from the fog of her fading memory, a                              │
│  triumph over the terrible disease that threatens to steal her life's work."                                    │
│                                                                                                                 │
│                                                                                                                 │
│      Perform sentiment analysis and recommend soundtrack genres.                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🔧 Tool Execution Started (#2) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: script_sentiment_analyzer                                                                                │
│  Args: {'script_text': "Each note brings both joy and sadness as Margaret's fingers dance across the piano      │
│  keys. The wonderful melody emerges from the fog of her fading memory, a triumph over the terrible di...        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: Analyze this movie concept:                                                                              │
│                                                                                                                 │
│  Title: "The Last Symphony"                                                                                     │
│  Genre: Drama                                                                                                   │
│  Logline: An elderly composer, losing her memory to dementia, races to complete her                             │
│  final masterpiece while her estranged daughter learns to understand her through music.                         │
│                                                                                                                 │
│  Script Excerpt: "Each note brings both joy and sadness as Margaret's fingers dance across                      │
│  the piano keys. The wonderful melody emerges from the fog of her fading memory, a                              │
│  triumph over the terrible disease that threatens to steal her life's work."                                    │
│                                                                                                                 │
│                                                                                                                 │
│      Perform sentiment analysis and recommend soundtrack genres.                                                │
│  Agent: Script Analyst                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 36ec3902-c762-4451-b5ca-4fe30fbe35c6                                                                       │
│  Final Output: Sentiment Analysis:                                                                              │
│  The script excerpt presents a predominantly uplifting and positive emotional tone, with a sentiment breakdown  │
│  showing 60% positive and 40% negative tones. The positive emotions arise from the joy and beauty in            │
│  Margaret's music, symbolizing triumph and hope despite the sadness and challenge of her fading memory due to   │
│  dementia. There is no suspenseful tone detected, emphasizing the drama's emotional and heartfelt nature.       │
│                                                                                                                 │
│  Soundtrack Recommendations:                                                                                    │
│  For a drama with an emotional mood, the recommended soundtrack genres are:                                     │
│  - Piano Solo: Reflects the intimate and personal nature of Margaret's music.                                   │
│  - String Quartet: Adds depth and emotional layers, enhancing poignant moments.                                 │
│  - Acoustic Guitar: Brings warmth and a human touch, complementing the piano melodies.                          │
│                                                                                                                 │
│  A combination of Piano Solo with Acoustic Guitar is suggested to create dynamic and emotionally resonant       │
│  scenes, supporting the themes of memory, legacy, and reconciliation in the story.                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✅ Execution complete with observability tracking!


╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are available!                                │
│                                                                             │
│  View insights including:                                                   │
│    • Agent decision-making process                                          │
│    • Task execution flow and timing                                         │
│    • Tool usage details                                                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
Would you like to view your execution traces? [y/N] (20s timeout): 

┌───────────────────────── Tracing Preference Saved ──────────────────────────┐
│                                                 

## 📈 Step 12: Analyze Observability Data

Let's examine the metrics and logs collected during execution.

In [14]:
# Display metrics
print(observer.get_metrics())

# Display all logs
observer.display_logs()

# Show result
print("\n📋 AGENT OUTPUT")
print("=" * 80)
print(result_observed)
print("=" * 80)


📊 EXECUTION METRICS
⏱️  Total Duration: 6.18 seconds
📝 Total Events Logged: 7
🤖 Agent Activity:
   • Script Analyst: 3 events
   • Budget Manager: 1 events
   • Market Researcher: 1 events

📋 EXECUTION LOGS (7 entries)
[10:00:33] CREW_START      | System                    | {'message': 'Crew execution started'}
[10:00:33] AGENT_INIT      | Script Analyst            | {'tools': ['sentiment_analyzer', 'soundtrack_recommender']}
[10:00:33] AGENT_INIT      | Budget Manager            | {'tools': ['budget_calculator']}
[10:00:33] AGENT_INIT      | Market Researcher         | {'tools': ['market_data_mcp']}
[10:00:33] TASK_START      | Script Analyst            | {'task': 'Script analysis'}
[10:00:40] TASK_COMPLETE   | Script Analyst            | {'status': 'success'}
[10:00:40] CREW_END        | System                    | {'message': 'Crew execution completed', 'duration_seconds': 6.18}

📋 AGENT OUTPUT
Sentiment Analysis:
The script excerpt presents a predominantly uplifting and positive 

## Step 13: Advanced Observability - Export Logs

In production, you'd want to export logs for analysis. Here's how to save them.

In [15]:
import json

# Export logs to JSON
def export_logs(observer, filename='crew_execution_logs.json'):
    """Export observability logs to a JSON file"""
    log_data = {
        'execution_summary': {
            'start_time': datetime.fromtimestamp(observer.start_time).isoformat(),
            'end_time': datetime.fromtimestamp(observer.end_time).isoformat(),
            'duration_seconds': round(observer.end_time - observer.start_time, 2),
            'total_events': len(observer.logs)
        },
        'events': observer.logs
    }

    with open(filename, 'w') as f:
        json.dump(log_data, f, indent=2)

    return f"✅ Logs exported to {filename}"

# Export the logs
print(export_logs(observer))

# Display a sample of the JSON structure
print("\n📄 Sample JSON structure:")
sample = {
    'execution_summary': 'metadata about the run',
    'events': [
        {
            'timestamp': '2025-11-30T10:30:45.123456',
            'event_type': 'TASK_START',
            'agent': 'Script Analyst',
            'details': {'task': 'Script analysis'}
        }
    ]
}
print(json.dumps(sample, indent=2))

✅ Logs exported to crew_execution_logs.json

📄 Sample JSON structure:
{
  "execution_summary": "metadata about the run",
  "events": [
    {
      "timestamp": "2025-11-30T10:30:45.123456",
      "event_type": "TASK_START",
      "agent": "Script Analyst",
      "details": {
        "task": "Script analysis"
      }
    }
  ]
}


## 🎯 CHALLENGE: Build Your Own Movie Production Crew!

Now it's your turn to apply what you've learned! Complete the following challenge:

### 🎬 Challenge Description

Create a **Movie Casting Assistant** crew that helps with casting decisions for a new film.

### Requirements:

1. **Create TWO custom tools:**
   - `actor_availability_checker`: Takes an actor name and date range, returns if they're available (simulate with logic)
   - `role_matcher`: Takes character description and returns suggested actor traits (age range, experience level, acting style)

2. **Create ONE MCP-style tool:**
   - `fetch_actor_database`: Simulates querying a database of actors with their filmography, awards, and availability

3. **Create TWO agents:**
   - **Casting Director**: Uses role_matcher and fetch_actor_database
   - **Schedule Coordinator**: Uses actor_availability_checker

4. **Define tasks:**
   - Task 1: Find suitable actors for a lead role (give a character description)
   - Task 2: Check availability of suggested actors for specific shooting dates

5. **Add observability:**
   - Use the CrewObserver to track execution
   - Export logs at the end

### 💡 Bonus Points:
- Make your character descriptions creative and unique
- Add personality to your agents' backstories
- Include error handling in your tools
- Create a visualization of the observability data

### Sample Character (feel free to create your own!):
```
Character: Captain Zara Quinn
Description: A 35-year-old space captain with a mysterious past, known for her
quick wit and exceptional piloting skills. Must convey both strength and vulnerability.
```

### 📝 Your Code Goes Below:

Good luck! 🚀

In [ ]:
# YOUR CHALLENGE CODE HERE
import datetime
# Step 1: Create your custom tools
# Hint: Use @tool decorator
@tool
def actor_availability_checker(actor_name: str, date_range: str) -> str:
    """
    Checks if actor is available for a date range.
    Date range format: 'YYYY-MM-DD to YYYY-MM-DD'
    """
    start_str, end_str = date_range.split(" to ")
    start_date = datetime.datetime.strptime(start_str, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_str, "%Y-%m-%d")

    if end_date < start_date:
        return "End date cant before start date."

    unavailable_actors = ["Scarlett Blaze", "John Titan"]

    if actor_name in unavailable_actors:
        return f"{actor_name} is NOT available between {date_range}."

    return f"{actor_name} is available between {date_range}."

@tool
def role_matcher(character_description: str) -> str:
    """
    Analyzes character description and suggests actor traits.
    """

    traits = {
        "age_range": "30-40",
        "experience_level": "Highly experienced (10+ years)",
        "acting_style": "Emotionally dynamic and charismatic"
    }

    if "space" in character_description.lower():
        traits["acting_style"] = "Commanding, intelligent, subtle vulnerability"

    if "villain" in character_description.lower():
        traits["acting_style"] = "Dark, intense, psychologically layered"

    return f"Suggested Traits: {traits}"



# Step 2: Create your MCP-style tool

@tool
def fetch_actor_database(traits: str) -> str:
    """
    Simulates querying an actor database.
    """

    mock_database = [
        {
            "name": "Lena Starfall",
            "age": 36,
            "experience": 12,
            "awards": 3,
            "filmography": ["Galactic Wars", "Silent Horizon"]
        },
        {
            "name": "Marcus Steele",
            "age": 39,
            "experience": 15,
            "awards": 1,
            "filmography": ["Dark Orbit", "The Last Colony"]
        },
        {
            "name": "Scarlett Blaze",
            "age": 34,
            "experience": 11,
            "awards": 5,
            "filmography": ["Solar Queen", "Edge of Infinity"]
        }
    ]

    results = [actor for actor in mock_database if 30 <= actor["age"] <= 40]

    if not results:
        return "No matching actors found."

    return f"Matching Actors: {results}"

# Step 3: Create your agents
casting_director = Agent(
    role="Casting Director",
    goal="Find the perfect lead actor for the film",
    backstory=(
        "A visionary casting expert known for discovering unexpected stars. "
        "Believes the soul of the character must match the soul of the actor."
    ),
    tools=[role_matcher, fetch_actor_database],
    verbose=True
)

schedule_coordinator = Agent(
    role="Schedule Coordinator",
    goal="Ensure actors are available for filming dates",
    backstory=(
        "A logistics mastermind who lives by spreadsheets and calendar alerts. "
        "No double bookings escape their radar."
    ),
    tools=[actor_availability_checker],
    verbose=True
)


# Step 4: Define your tasks
character_description = """
Captain Zara Quinn:
A 35-year-old space captain with a mysterious past.
Sharp-witted, emotionally complex, fearless leader.
Must portray strength and hidden vulnerability.
"""

task1 = Task(
    description=f"""
    Analyze the following character and suggest suitable actors.

    {character_description}
    """,
    expected_output="""
    A list of 2-3 suitable actors including:
    - Name
    - Age
    - Experience
    - Why they match the role
    """,
    agent=casting_director
)

task2 = Task(
    description="""
    Check availability of all suggested actors
    for shooting dates: 2026-06-01 to 2026-08-30
    """,
    expected_output="""
    Availability report for each actor:
    - Available or Not Available
    - Explanation
    """,
    agent=schedule_coordinator
)

# Step 5: Set up observability

crew = Crew(
    agents=[casting_director, schedule_coordinator],
    tasks=[task1, task2],
    observer=observer,
    verbose=True
)

result = crew.kickoff()

print("FINAL RESULT:\n")
print(result)

# logs = observer.get_logs()
# print("\Execution Logs:")
# print(logs)

# Step 6: Create and run your crew


# Step 7: Display results and metrics

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: bd6121db-d598-4f2d-a6e0-914e3c3d2fad                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│      Analyze the following character and suggest suitable actors.                                               │
│                                                                                                                 │
│                                                                                                                 │
│  Captain Zara Quinn:                                                                                            │
│  A 35-year-old space captain with a mysterious past.                                                            │
│  Sharp-witted, emotionally complex, fearless leader.                                                            │
│  Must portray strength and hidden vulnerability.                                                                │
│                                                                                                                 │
│                                                                                                                 │
│  ID: 1f5cb7dd-c665-43d6-b535-f0f3384fe81f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Casting Director                                                                                        │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze the following character and suggest suitable actors.                                               │
│                                                                                                                 │
│                                                                                                                 │
│  Captain Zara Quinn:                                                                                            │
│  A 35-year-old space captain with a mysterious past.                                                            │
│  Sharp-witted, emotionally complex, fearless leader.                                                            │
│  Must portray strength and hidden vulnerability.                                                                │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Tool role_matcher executed with result: Suggested Traits: {'age_range': '30-40', 'experience_level': 'Highly experienced (10+ years)', 'acting_style': 'Commanding, intelligent, subtle vulnerability'}...


╭──────────────────────────────────────── 🔧 Tool Execution Started (#3) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: role_matcher                                                                                             │
│  Args: {'character_description': 'Captain Zara Quinn: A 35-year-old space captain with a mysterious past.       │
│  Sharp-witted, emotionally complex, fearless leader. Must portray strength and hidden vulnerability.'}          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│      Check availability of all suggested actors                                                                 │
│      for shooting dates: 2026-06-01 to 2026-08-30                                                               │
│                                                                                                                 │
│  Agent: Schedule Coordinator                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: bd6121db-d598-4f2d-a6e0-914e3c3d2fad                                                                       │
│  Final Output: Availability report for each actor:                                                              │
│                                                                                                                 │
│  1. Lena Starfall                                                                                               │
│     - Available                                                                                                 │
│     - Explanation: Lena Starfall is available for the entire shooting period from 2026-06-01 to 2026-08-30,     │
│  ensuring she can fully commit to the project.                                                                  │
│                                                                                                                 │
│  2. Marcus Steele                                                                                               │
│     - Available                                                                                                 │
│     - Explanation: Marcus Steele is also available throughout the specified shooting dates, making him a        │
│  reliable choice for the role.                                                                                  │
│                                                                                                                 │
│  3. Scarlett Blaze                                                                                              │
│     - Not Available                                                                                             │
│     - Explanation: Scarlett Blaze has scheduling conflicts and is not available for filming between 2026-06-01  │
│  and 2026-08-30, which may require reconsidering her casting or adjusting the shooting schedule.                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

FINAL RESULT:

Availability report for each actor:

1. Lena Starfall
   - Available
   - Explanation: Lena Starfall is available for the entire shooting period from 2026-06-01 to 2026-08-30, ensuring she can fully commit to the project.

2. Marcus Steele
   - Available
   - Explanation: Marcus Steele is also available throughout the specified shooting dates, making him a reliable choice for the role.

3. Scarlett Blaze
   - Not Available
   - Explanation: Scarlett Blaze has scheduling conflicts and is not available for filming between 2026-06-01 and 2026-08-30, which may require reconsidering her casting or adjusting the shooting schedule.


╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are available!                                │
│                                                                             │
│  View insights including:                                                   │
│    • Agent decision-making process                                          │
│    • Task execution flow and timing                                         │
│    • Tool usage details                                                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
Would you like to view your execution traces? [y/N] (20s timeout): 

┌───────────────────────── Tracing Preference Saved ──────────────────────────┐
│                                                 

## 🎉 Congratulations!

You've completed the CrewAI practice notebook! You've learned:

✅ How to create custom tools for agents  
✅ How to implement MCP-style tools for external integrations  
✅ How to configure and use Gemini 2.5 Flash model  
✅ How to create specialized agents with specific roles  
✅ How to define and orchestrate tasks  
✅ How to implement observability and monitoring  
✅ How to export and analyze execution logs  

### 🔗 Useful Resources:

- [CrewAI Documentation](https://docs.crewai.com/)
- [Google AI Studio](https://makersuite.google.com/)
- [LangChain Tools](https://python.langchain.com/docs/modules/agents/tools/)

Happy coding! 🚀🎬